# Create a slab with specified termination

Create a slab with a specified termination from a bulk material. 

<h2 style="color:green">Usage</h2>


1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. if prompted, select the termination for the slab in cell 2.5.
1. Scroll down to view results. 


## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up defect parameters 

In [ ]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False 

MILLER_INDICES = (0, 0, 1)
THICKNESS = 3  # in atomic layers
VACUUM = 10.0  # in angstroms
XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
USE_ORTHOGONAL_Z = True
USE_CONVENTIONAL_CELL = True

# Index of the termination to be selected
TERMINATION_INDEX = 0

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
  
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("")

### 1.3. Get input materials

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
material = materials[0]

### 1.4. Preview the material

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize(material, repetitions=[3, 3, 1], rotation="0x")
visualize(material, repetitions=[3, 3, 1], rotation="-90x")

## 2. Configure slab and select termination

### 2.1. Create slab configuration
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.


In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration

slab_configuration = SlabConfiguration(
    bulk=material,
    miller_indices=MILLER_INDICES,
    thickness=THICKNESS, # in atomic layers
    vacuum=VACUUM, # in angstroms
    xy_supercell_matrix=XY_SUPERCELL_MATRIX,
    use_orthogonal_z=USE_ORTHOGONAL_Z,
    use_conventional_cell=USE_CONVENTIONAL_CELL,
)

### 2.2. Get possible terminations for the slabs

In [ ]:
from mat3ra.made.tools.build.slab import get_terminations
slab_terminations = get_terminations(slab_configuration)

### 2.3. Visualize slabs for all possible terminations

In [ ]:
from mat3ra.made.tools.build.slab import create_slab

slabs = [create_slab(slab_configuration, termination) for termination in slab_terminations]

visualize([{"material":slab, "title": slab.metadata["build"]["termination"]} for slab in slabs ], repetitions=[3, 3, 1], rotation="-90x")

### 2.4. Print terminations for the Slab

In [ ]:
print("Terminations")
for idx, termination in enumerate(slab_terminations):
    print(f"    {idx}: {termination}")

### 2.5. Select termination for the Slab

In [ ]:
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

termination_index = TERMINATION_INDEX

termination = slab_terminations[termination_index]
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        termination = await ui_prompt_select_array_element_by_index_pyodide(slab_terminations, element_name="termination")
    else:
        termination = ui_prompt_select_array_element_by_index(slab_terminations, element_name="termination")

## 3. Create the slab with selected termination

In [ ]:
from mat3ra.made.tools.build.slab import create_slab

slab = create_slab(slab_configuration, termination)

## 4. Visualize the slab

In [ ]:
visualize(slab, repetitions=[1, 1, 1], rotation="0x")

# 5. Pass material to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials
set_materials(slab)